In [ ]:
from __future__ import division
import os
import numpy as np
import time
import matplotlib.pyplot as plt
import scipy as scp
import pylab as pyl

import warnings
warnings.filterwarnings('ignore')
np.random.seed(1234)

%matplotlib inline
%load_ext autoreload
%autoreload 

In [ ]:
if not os.path.isdir('Sinkhorn_images'):
    os.makedirs('Sinkhorn_images')

In [ ]:
"""To compute distance matrix"""
def distmat(x,y):
    return np.sum( x**2,0 )[:,None] + np.sum( y**2,0 )[None,:] - 2*x.transpose().dot(y)

"""To Normalise a vector"""
normalize = lambda a: a/np.sum( a )

"""To Compute P"""
def GetP(u,K,v):
    u = u.reshape(u.shape[0],)
    v = v.reshape(v.shape[0],)
    return u[:,None]*K*v[None,:]

def plotp(x, col,plt, scale=200, edgecolors="k"):
  return plt.scatter( x[0,:], x[1,:], s=scale, edgecolors=edgecolors,  c=col, cmap='plasma', linewidths=2 )

In [ ]:
import computational_OT

## Sinkhorn for varying N and fixed $\varepsilon$

In [ ]:
def randomsampledata(N):
  x = []
  y = []
  N=np.sort(N)
  for i in range(len(N)):
    x.append(np.random.rand(2,N[i])-0.5)
    theta= 2*np.pi*np.random.rand(1,N[i])
    r = 0.8+0.2*np.random.rand(1,N[i])
    y.append(np.vstack((np.cos(theta)*r,np.sin(theta)*r)))
  
  return x,y,N

In [ ]:
N = [ 200,400,600,800,1000 ]
x,y,N = randomsampledata(N)

In [ ]:
# Sinkhorn
print("Sinkhorn.... ")
SinkhornP = []
results_Sinkhorn = []
times_Sinkhorn = []
Pmatrix_dist_linVSsinkhorn = []
for i in range(len(N)):

  
  xi,yi = x[i],y[i]
  #Cost matrix
  C = distmat(xi,yi)
  
  # a and b
  a = normalize(np.ones(N[i]))
  a = a.reshape(a.shape[0],-1)
  b = normalize(np.ones(N[i]))
  b = b.reshape(b.shape[0],-1)
  
  #Epsilon
  epsilon = .06

  #Kernel
  K = np.exp(-C/epsilon)


  print("Doing for ",N[i])
  print( " |- Iterating")

  #Inflating
  u = a
  v = b

  start = time.time()
  Optimizer = computational_OT.Sinkhorn(K,a,b,u,v,epsilon)
  out = Optimizer._update()
  results_Sinkhorn.append(out)
  end = time.time()
  times_Sinkhorn.append(end-start)
  print( " |- Computing P")
  print( "" )
  SinkhornP.append(GetP(out['u'],K,out['v']))
  

In [ ]:
plt.figure(figsize = (20,7))

plt.subplot(2,1,1),
plt.title("$||P1 -a||_1$")
for result in results_Sinkhorn:
  plt.plot( np.asarray(result['error_a']), linewidth = 2)
plt.yscale( 'log')
plt.legend([str(i) for i in N],loc="upper right")

plt.subplot(2,1,2)
plt.title("$||P^T 1 -b||_1$")
for result in results_Sinkhorn:
  plt.plot( np.asarray(result['error_b']), linewidth = 2)
plt.yscale( 'log')
plt.legend([str(i) for i in N],loc="upper right")
plt.savefig("Sinkhorn_images/ConvergenceSinkhorn.png")
plt.show()

In [ ]:
plt.figure(figsize = (20,7))
plt.subplot(2,1,1),
plt.title("Objective Function")


for result in results_Sinkhorn:
  plt.plot(np.asarray(result['objectives']).flatten(), linewidth = 2)
plt.legend([str(i) for i in N],loc="upper right")
plt.savefig("Sinkhorn_images/ObjectivefunctionSinkhorn.png")
plt.show()

## Sinkhron for varying $\varepsilon$

In [ ]:
N = [ 400,500 ]

In [ ]:
x = np.random.rand( 2,N[0] )-0.5
theta = 2*np.pi*np.random.rand( 1,N[1] )
r = 0.8+.2*np.random.rand( 1,N[1] )
y = np.vstack( ( r*np.cos( theta ),r*np.sin( theta ) ) )

In [ ]:
# Sinkhorn
print("Sinkhorn.... ")
SinkhornP=[]
results_Sinkhorn=[]
times_Sinkhorn=[]
epsilons=[ 1.0, 0.75, 0.5, 0.4, 0.3, 0.1, 0.05,0.03 ]
Pmatrix_dist_linVSsinkhorn=[]
for eps in epsilons:

  
  #Cost matrix
  C = distmat(x,y)
  
  # a and b
  a = normalize(np.ones(N[0]))
  a = a.reshape(a.shape[0],-1)
  b = normalize(np.ones(N[1]))
  b = b.reshape(b.shape[0],-1)



  #Kernel
  K = np.exp(-C/eps)


  print( "Doing for (",N[0],N[1],")." )
  print( " |- Iterating" )

  #Inflating
  u = a
  v = b

  start = time.time()
  Optimizer = computational_OT.Sinkhorn( K,a,b,u,v,eps )
  out = Optimizer._update()
  results_Sinkhorn.append( out )
  end = time.time()
  times_Sinkhorn.append( end-start )
  print( " |- Computing P" )
  print( "" )
  SinkhornP.append( GetP( out['u'],K,out['v'] ) )
  

In [ ]:
plt.figure( figsize = (20,7) )

plt.subplot(2,1,1),
plt.title( "$||P1 -a||_1+||P1 -b||_1$" )
for i in range( len(results_Sinkhorn) ):
  error=np.asarray( results_Sinkhorn[i]['error_a'] )+np.asarray( results_Sinkhorn[i]['error_b'] )
  plt.plot( error,label='Sinkhorn for $\epsilon=$'+ str(epsilons[i]), linewidth = 2 )
plt.yscale( 'log' )
plt.legend()

plt.savefig("Sinkhorn_images/ConvergenceSinkhornvaryingepsilon.png")
plt.show()

## Log-domain Sinkhorn

In [ ]:
def mina_u(H,epsilon): return -epsilon*np.log( np.sum(a * np.exp(-H/epsilon),0) )
def minb_u(H,epsilon): return -epsilon*np.log( np.sum(b * np.exp(-H/epsilon),1) )

def mina(H,epsilon): return mina_u(H-np.min(H,0),epsilon) + np.min(H,0);
def minb(H,epsilon): return minb_u(H-np.min(H,1)[:,None],epsilon) + np.min(H,1);


In [ ]:
epsilon = .01

a = np.ones((N[0],1))/N[0]
b = np.ones((1,N[1]))/N[1]


x = np.random.rand(2,N[0])-.5
theta = 2*np.pi*np.random.rand(1,N[1])
r = .8 + .2*np.random.rand(1,N[1])
y = np.vstack( (np.cos(theta)*r,np.sin(theta)*r) )
C = distmat(x,y)

In [ ]:
def Sinkhorn(C,epsilon,f,niter = 500):    
    Err = np.zeros(niter)
    for it in range(niter):
        g = mina(C-f[:,None],epsilon)
        f = minb(C-g[None,:],epsilon)
        # generate the coupling
        P = a * np.exp((f[:,None]+g[None,:]-C)/epsilon) * b
        # check conservation of mass
        Err[it] = np.linalg.norm(np.sum(P,0)-b,1)
    return (P,Err)
# run with 0 initialization for the potential f
(P,Err) =  Sinkhorn(C,epsilon,np.zeros(N[0]),3000)  
plt.plot(np.log10(Err));

### For varying $\varepsilon$

In [ ]:
t = []
epsilons = [ .1, .05, .01, .001 ]
for epsilon in epsilons:
    start = time.time()    
    (P,Err) =  Sinkhorn(C,epsilon,np.zeros(N[0]),1000)  
    end = time.time()
    t.append( 1e3*(end-start) )
    plt.plot( np.log10(Err), label='$\epsilon=$' + str(epsilon ) ) 
plt.legend()
plt.xlabel( 'Number of iterations' )
plt.ylabel( 'log10 error' )
plt.show()

#### Time Plot

In [ ]:
plt.figure( figsize = (15,8) )
plt.plot( epsilons, t, linestyle='--', marker='o', color='b' ,linewidth = 2 )
plt.xlabel( '$\epsilon$' ,fontsize = 18 )
plt.ylabel( 'Time in ms' )
plt.show()